In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd
import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re

In [2]:
# Naver API Key 가져오기
with open('naver_api_key.txt', 'r') as file:
    lines = file.readlines()
    client_id = lines[0].strip()
    client_secret = lines[1].strip()

In [3]:
# 저장한 파일 가져오기
kosdaq_day_price_path = './data/day_price/kosdaq'
kospi_day_price_path = './data/day_price/kospi'

kospi_xlsx_files = glob.glob(os.path.join(kospi_day_price_path, '*.xlsx'))
kospi_file_names = [os.path.basename(file) for file in kospi_xlsx_files]

kosdaq_xlsx_files = glob.glob(os.path.join(kosdaq_day_price_path, '*.xlsx'))
kosdaq_file_names = [os.path.basename(file) for file in kosdaq_xlsx_files]

In [14]:
# test
get_title_keyword(kosdaq_file_names)
# kosdaq, 515 / 1600, 삼영에스앤씨, 뉴스 : 133 건 (24.07.29)
# kosdaq, 1357 / 1600, 태웅로직스, 뉴스 : 74 건 (24.07.30) 키워드 산출에 시간이 많이 소요되는 듯, 다른 키워드 추출모델 있는지 고려해봐야 될 듯

kosdaq, 516 / 1600, 삼영엠텍, 뉴스 : 85 건
kosdaq, 517 / 1600, 삼영이엔씨, 뉴스 : 105 건
kosdaq, 518 / 1600, 삼일, 뉴스 : 227 건
kosdaq, 519 / 1600, 삼일기업공사, 뉴스 : 59 건
kosdaq, 520 / 1600, 삼지전자, 뉴스 : 62 건
kosdaq, 521 / 1600, 삼진, 뉴스 : 269 건
kosdaq, 522 / 1600, 삼진엘앤디, 뉴스 : 130 건
kosdaq, 523 / 1600, 삼천당제약, 뉴스 : 179 건
kosdaq, 524 / 1600, 삼천리자전거, 뉴스 : 169 건
kosdaq, 525 / 1600, 삼표시멘트, 뉴스 : 39 건
kosdaq, 526 / 1600, 삼현, 뉴스 : 51 건
kosdaq, 527 / 1600, 삼현철강, 뉴스 : 27 건
kosdaq, 528 / 1600, 삼화네트웍스, 뉴스 : 105 건
kosdaq, 529 / 1600, 상보, 뉴스 : 131 건
kosdaq, 530 / 1600, 상상인, 뉴스 : 96 건
kosdaq, 531 / 1600, 상신이디피, 뉴스 : 63 건
kosdaq, 532 / 1600, 상신전자, 뉴스 : 95 건
kosdaq, 533 / 1600, 상아프론테크, 뉴스 : 123 건
kosdaq, 534 / 1600, 상지건설, 뉴스 : 2 건
kosdaq, 535 / 1600, 새로닉스, 뉴스 : 49 건
kosdaq, 536 / 1600, 새빗켐, 뉴스 : 59 건
kosdaq, 537 / 1600, 샌즈랩, 뉴스 : 33 건
kosdaq, 538 / 1600, 샘씨엔에스, 뉴스 : 34 건
kosdaq, 539 / 1600, 샤페론, 뉴스 : 41 건
kosdaq, 540 / 1600, 서남, 뉴스 : 65 건
kosdaq, 541 / 1600, 서린바이오, 뉴스 : 119 건
kosdaq, 542 / 1600, 서부T&D, 뉴스 : 85 건
kosdaq, 543 / 160

KeyboardInterrupt: 

In [17]:
# test2
# kosdaq, 15 / 1600, CS, 뉴스 : 6440 건
# 목적 : 주식 CS 같은 대량으로 검색되는 뉴스 목록 줄이기 
# test 결과 : 성공
# kosdaq, 1 / 1600, cs, 뉴스 : 131 건
get_title_keyword(kosdaq_file_names)

kosdaq, 1 / 1600, cs, 뉴스 : 131 건


In [13]:
def get_title_keyword(market_list):
    if market_list == kospi_file_names:
        market_name = "kospi"
    elif market_list == kosdaq_file_names:
        market_name = "kosdaq"
    
    save_path = "data/news_and_keyword/" + market_name + "/"
    
    #for i in range(len(market_list)):
    #for i in range(1):
    for i in range(515,len(market_list)):
        stock_name = market_list[i].replace(".xlsx","")
        #stock_name = '3S'
        #stock_name = 'cs'
        node = 'news'  # 크롤링 할 대상
        src_text = '[특징주] ' + stock_name
        cnt = 0
        title_result = ""
        title_result_list = []
        date_result_list = []
        keyword_result_list = []
        rate_d1_list = []
        rate_d5_list = []
        rate_d20_list = []

        jsonResponse = get_naver_search(node, src_text, 1, 100)  # [CODE 2]
        total = jsonResponse['total']

        while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
            for post in jsonResponse['items']:
                cnt += 1
                title_result, date_result = get_post_data(stock_name, post)  # [CODE 3]
                if title_result == "":
                    continue

                keywords = get_keyword(title_result, stock_name)

                if len(keywords) > 0:
                    rate_d1, rate_d5, rate_d20 = get_rate(market_name, stock_name, date_result)

                    title_result_list.append(title_result)
                    date_result_list.append(date_result)
                    keyword_result_list.append(keywords)
                    rate_d1_list.append(rate_d1)
                    rate_d5_list.append(rate_d5)
                    rate_d20_list.append(rate_d20)

            start = jsonResponse['start'] + jsonResponse['display']
            # print(jsonResponse['display'])
            time.sleep(0.1)
            jsonResponse = get_naver_search(node, src_text, start, 100)  # [CODE 2]
        
        if len(title_result_list) > 0:
            df_news_keyword = pd.DataFrame(list(zip(date_result_list, title_result_list, keyword_result_list, rate_d1_list, rate_d5_list, rate_d20_list)))
            df_news_keyword.columns = ["Date", "Title", "Keywords", "Rate_D1", "Rate_D5", "Rate_D20"]
            df_news_keyword.to_excel(save_path + stock_name + ".xlsx", index= False)
        
        print('{}, {} / {}, {}, 뉴스 : {} 건'.format(market_name, i+1, len(market_list), stock_name, total))

In [5]:
# [CODE 1]
def get_request_url(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            #print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [6]:
# [CODE 2]
def get_naver_search(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    #parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    parameters = '?query="{}"&start={}&display={}'.format(urllib.parse.quote(srcText), start, display) # 정확이 만족하는 부분만 검색하도록 query="{}" 부분 처리

    url = base + node + parameters
    responseDecode = get_request_url(url)  # [CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [7]:
# [CODE 3]
def get_post_data(stock_name, post):
    filter_pattern = r"\d{1,3}(,\d{3})?원, \d+일 \d+시 \d+분"

    title_result = ""
    date_result = ""
    
    title = post['title']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate2 = pDate.strftime('%Y-%m-%d %H:%M:%S')
    if pDate >= datetime.datetime.strptime('2015-06-15', '%Y-%m-%d'):
        title = title.replace("<b>","")
        title = title.replace("</b>", "")

        src_text = "[특징주] " + stock_name
        src_text2 = "[특징주]" + stock_name

        if src_text in title or src_text2 in title:
            title_result = title
            date_result = pDate2
            title_result = title_result.replace("[특징주]","")
            title_result = title_result.replace("&quot;","")
            title_result = title_result.replace(stock_name,"")
            title_result = title_result.lstrip()
            
            if title_result.startswith(", "):
                title_result = title_result[2:]

            pattern_match = re.search(filter_pattern, title_result)
            if pattern_match:
                title_result = ""
            
    return title_result, date_result

In [8]:
def get_keyword(title, stock_name): # 종목명과 기사제목을 받아 키워드를 추출함
    # 직접 정의한 한국어 불용어 목록
    stop_words = set([
        '이', '그', '저', '것', '수', '등', '들', '및', '의', '를', '에', '과', '와', '한', 
        '하다', '되다', '을', '를', '은', '는', '이', '가', '로', '으로', '에게', '께', '에서',
        '이다', '하다', '것', '있다', '되다', '수', '나', '우리', '너', '있다', '이다', '것', '특징주', '&quot;'
    ])
    
    stop_words.add(stock_name) # 불용어 목록에 종목명 추가(기사에 포함된 종목명이 키워드에 걸리지 않기 위해)

    words = word_tokenize(title)

    filtered_words = [word for word in words if word.isalnum() and word not in stop_words] # 불용어 및 특수문자 제거
    word_freq = Counter(filtered_words) # 단어 빈도 계산
    keywords = [word for word, freq in word_freq.most_common(5)] # 키워드 추출 (빈도순 상위 5개 단어)
    return keywords

In [9]:
def get_rate(market_name, stock_name, date):
    rate_d1 = -400
    rate_d5 = -400
    rate_d20 = -400
    
    load_path = "./data/day_price/" + market_name + "/" + stock_name + ".xlsx"
    df_stock_price = pd.read_excel(load_path)
    df_stock_price["Date"] = df_stock_price["Date"].dt.strftime("%Y-%m-%d")
    
    base_index = df_stock_price.index[df_stock_price["Date"].str.contains(str(date)[0:10])].tolist()
    base_close = df_stock_price["Close"][base_index]

    if len(base_index) == 0:
        return rate_d1, rate_d5, rate_d20  # 날짜가 없는 경우 기본값 반환
    
    base_index = base_index[0]
    base_close = df_stock_price["Close"].iloc[base_index]
    
    if (base_index + 20) < len(df_stock_price):
        d1_close = df_stock_price["Close"].iloc[base_index + 1]
        d5_close = df_stock_price["Close"].iloc[base_index + 5]
        d20_close = df_stock_price["Close"].iloc[base_index + 20]
        
        rate_d1 = (d1_close - base_close) / base_close * 100
        rate_d5 = (d5_close - base_close) / base_close * 100
        rate_d20 = (d20_close - base_close) / base_close * 100
        
    return rate_d1, rate_d5, rate_d20